In [1]:
from data_utils import *
from dataset import TLDataset
from transformers import T5ForConditionalGeneration,AutoTokenizer, AutoConfig
from torch.utils.data import Dataset, DataLoader
from finetuner import T5FineTuner
import torch 
from evaluate import *
from multiprocessing import cpu_count
import argparse
output_path_dir = "/home/salzubi_umass_edu/CKPTS/OAI_TRG_MODEL_CKPT/"
train_bsz = 4
lr = 1e-3
train_fraction = 1
model_name = "google/flan-t5-base"
file_name = f"OAI_trg_MODEL_task_Adam_lr:{lr}_bsz:{train_bsz}_frac:{train_fraction}_{model_name}"
num_gpus = torch.cuda.device_count()
num_cpus = cpu_count()

args_dict = dict(
    # file_name = file_name,
    # output_dir=output_path_dir, # path to save the checkpoints
    model_name_or_path=model_name,
    tokenizer_name_or_path=model_name,
    max_seq_length=512,
    train_fraction= train_fraction,
    learning_rate=lr,
    weight_decay=0.0,
    scheduler_factor = .1,
    use_gpu = True if num_gpus > 0 else False,
    train_batch_size=train_bsz,
    eval_batch_size=32,
    num_train_steps=10000000,
    es_patience = 4,
    val_check_interval = .25,
    dropout = .2,
    n_gpu=num_gpus,
    cpu_per_device=num_cpus,
    task = "classification",
)

args = argparse.Namespace(**args_dict)
tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path)


/work/salzubi_umass_edu/.conda/envs/wmt_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-02 14:10:52.290817: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-02 14:10:52.476117: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-02 14:10:54.086552: W tensorflow/compiler/xla/stream_execut

In [1]:
pwd

'/work/salzubi_umass_edu/T5_human_eval_finetune'

In [ ]:
from evaluate import *
from sklearn.metrics import accuracy_score, r2_score, mean_squared_error
import glob
trg_task = [35]
test_split = .2
fixed_path = "../experiments/"
# model_path = "SRC_task_[3]___TRG_task___[8]"
model_path = "SRC_task_[50, 33, 40, 32, 31, 51, 39, 52, 53, 57, 64, 54, 55, 14]___TRG_task___[35]"

scores = []

for i in range(1,4):
    random_seed = i
    ckpt_path = f"{fixed_path}{model_path}/trial_{i}/{model_path}___trial_{i}-v1.ckpt"
    # eval_path = f"{fixed_path}{model_path}/trial_{i}/{model_path}___trial_{i}_eval.csv"
    # pred_df = pd.read_csv(eval_path)
    train, val, test = generate_normal_splits(data_path = "/work/salzubi_umass_edu/human_eval_datasets/master_dataset.csv", random_state=random_seed,  trg_task_ids= trg_task,  split_size=test_split, source_tuning=False)
    test_model = load_ckpt(args, ckpt_path)
    test_data = TLDataset(test, tokenizer)
    test_loader = DataLoader(test_data, batch_size=args.eval_batch_size, num_workers=num_cpus) #, num_workers=num_cpus//num_gpus
    pred_df = evaluate_batch(test_loader, test_model, tokenizer)
    scores += [accuracy_score(pred_df.predicted, pred_df.ground_truth)]
    # scores += [mean_squared_error(pred_df.predicted, pred_df.ground_truth)]


In [3]:
scores

[0.7148936170212766, 0.7319148936170212, 0.7382978723404255]

In [4]:
import numpy as np

print(np.mean(scores))
print()
print(np.std(scores))

0.728368794326241

0.009878289558286592


In [7]:
with open(f"{fixed_path}{model_path}/trial_{i}/FINAL_EVAL.txt", "w") as file:
    file.write(f"The {i} different accuracies are: {scores}\n")
    file.write(f"\nAVG ACC: {np.mean(scores)}\n")
    file.write(f"STD: {np.std(scores)}")

In [ ]:
from data_utils import *

train, val = generate_normal_splits(data_path = "/work/salzubi_umass_edu/human_eval_datasets/master_dataset.csv", random_state=1,
                                    trg_task_ids=[24,25,26],
                                    train_fraction=1, source_tuning=True)

In [3]:
from data_utils import *
from evaluate import *
from dataset import TLDataset
from transformers import T5ForConditionalGeneration,AutoTokenizer, AutoConfig
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import argparse
import logging
import torch
from multiprocessing import cpu_count
from finetuner import T5FineTuner
from ray_lightning import RayStrategy
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from tqdm import tqdm
from pytorch_lightning import loggers as pl_loggers
import ray
import ast
import os
import gc


output_path_dir = "/home/salzubi_umass_edu/CKPTS/OAI_TRG_MODEL_CKPT/"
train_bsz = 4
lr = 1e-3
train_fraction = 1
model_name = "google/flan-t5-base"
file_name = f"OAI_trg_MODEL_task_Adam_lr:{lr}_bsz:{train_bsz}_frac:{train_fraction}_{model_name}"
num_gpus = torch.cuda.device_count()
num_cpus = cpu_count()

args_dict = dict(
    # file_name = file_name,
    # output_dir=output_path_dir, # path to save the checkpoints
    model_name_or_path=model_name,
    tokenizer_name_or_path=model_name,
    max_seq_length=512,
    train_fraction= train_fraction,
    lr=lr,
    weight_decay=0.0,
    scheduler_factor = .1,
    use_gpu = True if num_gpus > 0 else False,
    train_batch_size=train_bsz,
    eval_batch_size=32,
    num_train_steps=10000000,
    es_patience = 4,
    val_check_interval = .25,
    dropout = .2,
    n_gpu=num_gpus,
    cpu_per_device=num_cpus,
    task = "classification",
)

args = argparse.Namespace(**args_dict)
tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path)


In [2]:
from data_utils import *

task_ids = [33,59,21]
train, val, _ = generate_normal_splits(data_path = "/work/salzubi_umass_edu/human_eval_datasets/master_dataset.csv", random_state=1,
                                       trg_task_ids=task_ids, val_split=.3, split_size=.3, train_fraction=1, source_tuning=True, example_threshold=50000)

train_data = TLDataset(train, tokenizer)
val_data = TLDataset(val, tokenizer)

train_loader = DataLoader(train_data, batch_size=8, drop_last=True) # , num_workers=num_cpus//num_gpus, sampler = train_sampler
val_loader = DataLoader(val_data, batch_size=8) #, num_workers=num_cpus//num_gpus

In [8]:
from data_utils import *
import pandas as pd
from dataset import TLDataset

train = pd.read_csv("../experiments/mocha_tests/processed_mocha_train_k=100.csv")
val = pd.read_csv("../experiments/mocha_tests/processed_mocha_val.csv")



In [17]:
train_data = TLDataset(train, tokenizer)
val_data = TLDataset(val, tokenizer)

# train_loader = DataLoader(train_data, batch_size=8, drop_last=True) # , num_workers=num_cpus//num_gpus, sampler = train_sampler
val_loader = DataLoader(val_data, batch_size=8) #, num_workers=num_cpus//num_gpus

In [ ]:
for ex in val_loader:
    print(ex)

In [5]:
print(len(train_loader))

12


In [6]:
print(len(val_loader))

877


In [ ]:
ray.shutdown()
tb_logger = pl_loggers.TensorBoardLogger(save_dir=f"")
strategy = RayStrategy(num_workers=num_gpus, use_gpu=True if num_gpus > 0 else False, find_unused_parameters=False)
es = EarlyStopping(monitor="val_loss", mode="min", patience=25)
checkpoint_callback = ModelCheckpoint(monitor='val_loss', dirpath = "", filename = "DELETE", mode="min")
val_check_interval = args.val_check_interval


model = T5FineTuner(args)
trainer = pl.Trainer(strategy=strategy, callbacks = [es]) # strategy = strategy
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score
df = pd.read_csv("../experiments/SRC_task_[50, 33, 40, 32, 31, 51, 39, 52, 53, 57, 64, 54, 55]___TRG_task___None/trial_1/SRC_task_[50, 33, 40, 32, 31, 51, 39, 52, 53, 57, 64, 54, 55]___TRG_task___None___trial_1_eval.csv")

In [2]:
accuracy_score(df.predicted, df.ground_truth)

0.52